## 1. Keyword/Text search - Default(Standard) Analyzer

- SDK doc: https://learn.microsoft.com/en-us/python/api/azure-search-documents/azure.search.documents.searchclient?view=azure-python

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
)

In [3]:
import os
service_endpoint = os.getenv("AZSCH_ENDPOINT")  
credential = AzureKeyCredential(os.environ["AZSCH_KEY"])

print(service_endpoint)

https://ikaisch.search.windows.net


In [4]:
# Create a search index
def create_search_index(index_name):
    index_client = SearchIndexClient(
        endpoint=service_endpoint, credential=credential)
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="proverb", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True),
        SearchableField(name="meaning", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True),
        SearchableField(name="source", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True),
        SearchableField(name="type", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True),
    ]

    # Create the search index
    index = SearchIndex(name=index_name, fields=fields)
    result = index_client.create_or_update_index(index)
    print(f' {result.name} created')

In [5]:
index_name = "korproverb-keywordstd-index"
create_search_index(index_name)

 korproverb-keywordstd-index created


In [6]:
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

In [7]:
import pandas as pd
from tqdm import tqdm

In [8]:
df = pd.read_csv('./source/idioms.csv')
df.head()

,Description,Meaning,Source,Type
0,가갸 뒤 자〔뒷다리〕도 모른다,"글자를 모르는 사람을 비유적으로 이르는 말.,사리에 어두운 사람을 놀림조로 이르는 말.",고려대 한국어대사전,속담
1,가까운 남이 먼 일가보다 낫다,이웃끼리 서로 친하게 지내다 보면 먼 곳에 있는 일가보다 더 친하게 되어 서로 도우...,표준국어대사전,속담
2,가까운 데 집은 깎이고 먼 데 절은 비친다,가까운 데 있는 절이나 집은 자잘한 흠도 잘 드러나서 좋지 않아 보이고 먼 곳에 윤...,표준국어대사전,속담
3,가까운 무당보다 먼 데 무당이 영하다,흔히 사람은 자신이 잘 알고 가까이 있는 것보다는 잘 모르고 멀리 있는 것을 더 좋...,표준국어대사전,속담
4,가까운 집 며느리일수록 흉이 많다,늘 가까이 있고 잘 아는 사이일수록 상대편의 결점이 눈에 더 많이 띈다는 말.,표준국어대사전,속담


In [9]:
count = 0
batch_size = 40
for i in tqdm(range(0, len(df), batch_size)):
    # set end position of batch
    i_end = min(i+batch_size, len(df))
    
    documents = df[i:i_end].apply(
        lambda row: {'id': str(row.name), 
                     'proverb': row['Description'], 
                     'meaning': row['Meaning'],
                     'source': row['Source'],
                     'type': str(row['Type'])
                    }, axis=1).to_list()
    
    result = search_client.upload_documents(documents)  

  0%|          | 0/164 [00:00<?, ?it/s]

100%|██████████| 164/164 [00:26<00:00,  6.13it/s]


In [10]:
from azure.search.documents.models import (
    QueryType,
    SearchMode
)

import json

def azsch_text_query(query, type=QueryType.Simple, mode=SearchMode.Any):

    results = search_client.search(  
        search_text=query,
        query_type=type,
        search_mode=mode,
        search_fields=["meaning"],
        select=["proverb", "meaning", "source", "type"],
        query_language="ko-kr",
        top=10 # for limiting text search
    ) 
    
    print("Search Results:")
    for i, result in enumerate(results, 1): 
        print(f"{i}: {result['@search.score']:.10f}: {result['proverb']} - {result['meaning']}, {result['type']}")  


## Basic search issues with Standard Analyzer

In [13]:
# Expected search results include '노력', '노력을', '노력하면', ...
results = azsch_text_query('노력')

Search Results:
1: 6.2826767000: 힘을 넣다 - 어떤 일을 이루려고 시간, 노력, 자금 따위를 투입하다., 관용구
2: 3.9428840000: 소불알 떨어지면 구워 먹겠다고 소금 가지고 따라다닌다 - 노력은 안 하고 산 소의 불알이 저절로 떨어지기를 마냥 기다리기만 한다는 뜻으로, 노력 없이 요행만 바라는 헛된 짓을 비유적으로 이르는 말., 속담


In [14]:
azsch_text_query('노력*')

Search Results:
1: 1.0000000000: 굴을 파야 금을 얻는다 - 목적을 이루기 위하여서는 거기에 필요한 조건을 갖추거나 노력을 하여야 함을 교훈적으로 이르는 말., 속담
2: 1.0000000000: 절뚝발이 말이 천리 간다 - 누구나 꾸준히 노력하면 무슨 일이든 이룰 수 있다는 말. (=둔한 말도 열흘 가면 천리를 간다.), 속담
3: 1.0000000000: 옥도 갈아야 빛이 난다 - 아무리 소질이 좋아도 이것을 잘 닦고 기르지 아니하면 훌륭한 것이 되지 못한다는 말.,고생을 겪으며 노력을 기울여야 뜻한 바를 이룰 수 있다는 말., 속담
4: 1.0000000000: 산에 가야 범을 잡는다 - 어떤 일을 이루기 위한 방향을 제대로 잡고 노력하여야만 그 일을 이룰 수 있음을 비유적으로 이르는 말.,무슨 일을 하려면 발 벗고 나서서 실제로 힘들여 해야 함을 비유적으로 이르는 말. (=산에 가야 꿩..., 속담
5: 1.0000000000: 옥도 갈아야〔닦아야〕 빛이 난다 - 아무리 소질이 좋아도 그것을 잘 수양하지 않으면 훌륭해질 수 없음을 이르는 말.,고생을 겪으면서 노력해야 뜻한 바를 이룰 수 있다는 말., 속담
6: 1.0000000000: 옥석도 닦아야 빛이 난다 - 아무리 소질이 좋아도 이것을 잘 닦고 기르지 아니하면 훌륭한 것이 되지 못한다는 말.,고생을 겪으며 노력을 기울여야 뜻한 바를 이룰 수 있다는 말., 속담
7: 1.0000000000: 빈손 털다 - (사람이) 들인 노력이 헛일이 되어 아무 소득이 없다.,(사람이) 가지고 있던 것을 몽땅 털다., 관용구
8: 1.0000000000: 쌀 한 알 보고 뜨물 한 동이 마신다 - 적은 이익을 위하여 노력이나 경비가 지나치게 많이 들어감을 비유적으로 이르는 말., 속담
9: 1.0000000000: 하늘에서  떨어지다 - (어떠한 성과가) 별로 노력을 들이지 않았는데 저절로 얻어지다., 관용구
10: 1.0000000000: 하늘을 보아야 별을 따지 - 어떤 성과를 거두려면 

In [16]:
azsch_text_query('+노력 +준비')

Search Results:


In [17]:
azsch_text_query('+노력* +준비*')

Search Results:
1: 2.0000000000: 하늘을 보아야 별을 따지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말.,무슨 일이 이루어질 기회나 조건이 전혀 없음을 이르는 말., 속담
2: 2.0000000000: 잠을 자야 꿈을 꾸지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말.,원인 없이 결과를 바랄 수 없음을 이르는 말., 속담
3: 2.0000000000: 새도 날려면 움츠린다 - 무슨 일이든 사전에 준비할 틈과 노력이 필요하다는 말. (=개구리도 옴쳐야 뛴다.), 속담
4: 2.0000000000: 눈을 떠야 별을 보지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말., 속담


In [18]:
azsch_text_query('+노력* -준비*')

Search Results:
1: 2.0000000000: 굴을 파야 금을 얻는다 - 목적을 이루기 위하여서는 거기에 필요한 조건을 갖추거나 노력을 하여야 함을 교훈적으로 이르는 말., 속담
2: 2.0000000000: 절뚝발이 말이 천리 간다 - 누구나 꾸준히 노력하면 무슨 일이든 이룰 수 있다는 말. (=둔한 말도 열흘 가면 천리를 간다.), 속담
3: 2.0000000000: 옥도 갈아야 빛이 난다 - 아무리 소질이 좋아도 이것을 잘 닦고 기르지 아니하면 훌륭한 것이 되지 못한다는 말.,고생을 겪으며 노력을 기울여야 뜻한 바를 이룰 수 있다는 말., 속담
4: 2.0000000000: 산에 가야 범을 잡는다 - 어떤 일을 이루기 위한 방향을 제대로 잡고 노력하여야만 그 일을 이룰 수 있음을 비유적으로 이르는 말.,무슨 일을 하려면 발 벗고 나서서 실제로 힘들여 해야 함을 비유적으로 이르는 말. (=산에 가야 꿩..., 속담
5: 2.0000000000: 옥도 갈아야〔닦아야〕 빛이 난다 - 아무리 소질이 좋아도 그것을 잘 수양하지 않으면 훌륭해질 수 없음을 이르는 말.,고생을 겪으면서 노력해야 뜻한 바를 이룰 수 있다는 말., 속담
6: 2.0000000000: 옥석도 닦아야 빛이 난다 - 아무리 소질이 좋아도 이것을 잘 닦고 기르지 아니하면 훌륭한 것이 되지 못한다는 말.,고생을 겪으며 노력을 기울여야 뜻한 바를 이룰 수 있다는 말., 속담
7: 2.0000000000: 빈손 털다 - (사람이) 들인 노력이 헛일이 되어 아무 소득이 없다.,(사람이) 가지고 있던 것을 몽땅 털다., 관용구
8: 2.0000000000: 쌀 한 알 보고 뜨물 한 동이 마신다 - 적은 이익을 위하여 노력이나 경비가 지나치게 많이 들어감을 비유적으로 이르는 말., 속담
9: 2.0000000000: 하늘에서  떨어지다 - (어떠한 성과가) 별로 노력을 들이지 않았는데 저절로 얻어지다., 관용구
10: 2.0000000000: 겉보리 한 말 주고 푸닥거리하기보다 낫다 - 들인 